In [84]:
#|default_exp losses

#|export
import torch
import numpy as np
import pandas as pd
from tsai.basics import *


# Custom losses

In [94]:
#|export
class Loss(nn.Module):
    def __init__(self, ranges, weights):
        super().__init__()
        self.register_buffer('ranges', torch.Tensor(ranges))
        self.register_buffer('weights', torch.Tensor(weights))

    def set_weights(self, weights):
        self.weights = torch.Tensor(weights).to(self.weights.device)

    def weighted_loss_tensor(self, target):        
        batch, variables, horizon = target.shape  # Shape (32, 4, 6)
        variable, range, interval = self.ranges.shape  # Shape (4, 4, 2)

        target_shaped = torch.reshape(target, (batch, variables, 1, horizon))  # Shape (32, 4, 6) -> (32, 4, 1, 6)
        ranges_shaped = torch.reshape(self.ranges, (variable, range, 1, interval))  # Shape (4, 4, 2) -> (4, 4, 1, 2)

        weights_tensor = ((ranges_shaped[..., 0] <= target_shaped) & (target_shaped <= ranges_shaped[..., 1])).float()
        
        return torch.einsum('r,bvrh->bvh', self.weights, weights_tensor)
    
    def loss_measure(self, y_pred, y_true):
        # Define the actual loss measure here
        return torch.abs(y_pred - y_true)  # Example: L1 loss
    
    def forward(self, y_pred, y_true):
        error = self.loss_measure(y_pred, y_true)
        weights = self.weighted_loss_tensor(y_true)
        loss = (error * weights).mean()
        
        return loss

In [95]:
# Test
device = 'cpu'
ranges = np.array([[[0, 1], [1, 2], [2, 3], [3, 4]],
                   [[0, 1], [1, 2], [2, 3], [3, 4]],
                   [[0, 1], [1, 2], [2, 3], [3, 4]],
                   [[0, 1], [1, 2], [2, 3], [3, 4]]])

weights = np.array([1, 2, 3, 4])

target = torch.tensor([[[0.5, 1.5, 2.5, 3.5, 4.5, 5.5],
                        [0.5, 1.5, 2.5, 3.5, 4.5, 5.5],
                        [0.5, 1.5, 2.5, 3.5, 4.5, 5.5],
                        [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]]], device=device, dtype=torch.float32)

input = target + 1

expected_weights = torch.tensor([[[1, 2, 3, 4, 0, 0],
                                 [1, 2, 3, 4, 0, 0],
                                 [1, 2, 3, 4, 0, 0],
                                 [1, 2, 3, 4, 0, 0]]], device=device, dtype=torch.float32)

def test_LossWeightsTensor():
    loss = Loss(ranges, weights).to(device)
    result = loss.weighted_loss_tensor(target)

    assert torch.equal(result, expected_weights), f"Expected {expected_weights}, but got {result}"
    print(f"Loss Tensor test passed!")

In [96]:
#|export
class wMSELoss(Loss):
    def __init__(self, ranges, weights):
        super().__init__(ranges, weights)

    
    def loss_measure(self, y_pred, y_true):
        return (y_true-y_pred)**2

In [97]:
#|export
class wMAELoss(Loss):
    def __init__(self, ranges, weights):
        super().__init__(ranges, weights)

    
    def loss_measure(self, y_pred, y_true):
        return torch.abs(y_true-y_pred)

In [98]:
def check_loss_function(loss_class, expected_value):
    loss = loss_class(ranges, weights).to(device)
    result = loss(input, target)

    assert torch.isclose(result, expected_value), f"Expected {expected_value}, but got {result}"
    print(f"{type(loss).__name__} test passed!")

def test_wMSELoss():
    expected_mse_loss = torch.mean(expected_weights * (input - target) ** 2)
    check_loss_function(wMSELoss, expected_mse_loss)

def test_wMAELoss():
    expected_mae_loss = torch.mean(expected_weights * torch.abs(input - target))
    check_loss_function(wMAELoss, expected_mae_loss)

In [99]:
#|export

class LossMetrics:
    def __init__(self, loss_func:Loss):
        self.loss_func = loss_func

    def loss_call(self, input, target, weight_idx):
        loss_copy = deepcopy(self.loss_func)

        weights = torch.zeros(len(loss_copy.weights))
        weights[weight_idx] = loss_copy.weights[weight_idx]
        loss_copy.set_weights(weights)
        
        return loss_copy.forward(input, target)

    def loss_low(self, input, target):
        return self.loss_call(input, target, 0)

    def loss_moderate(self, input, target):
        return self.loss_call(input, target, 1)

    def loss_elevated(self, input, target):
        return self.loss_call(input, target, 2)

    def loss_high(self, input, target):
        return self.loss_call(input, target, 3)

    def metrics(self):
        return [self.loss_low, self.loss_moderate, self.loss_elevated, self.loss_high]

In [100]:
# Test
def test_LossMetrics():
    loss = wMAELoss(ranges, weights).to(device)
    loss_metrics = LossMetrics(loss)

    metrics = loss_metrics.metrics()

    loss_value = loss(input, target)
    metrics_values = [metric(input, target) for metric in metrics]

    assert torch.isclose(loss_value, sum(metrics_values)), f"Expected {loss_value}, but got {sum(metrics_values)} ({metrics_values})"
    print("LossMetrics test passed!")

In [101]:
#| test
test_LossWeightsTensor()
test_wMSELoss()
test_wMAELoss()
test_LossMetrics()

Loss Tensor test passed!
wMSELoss test passed!
wMAELoss test passed!
LossMetrics test passed!
